In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import time

%config Completer.use_jedi = False 

In [3]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options=gpu_options))

### Load Data

In [4]:
X = np.load("X.npy")
y = np.load("y.npy")

### Normalize features.

In [5]:
X = tf.keras.utils.normalize(X)

## `Model`

In [20]:
NAME = "{}-conv-{}-nodes-{}-dense-{}".format(3, 128, 1, int(time.time()))
tensorboard = TensorBoard(log_dir='bcd_3logs')
print(NAME)

model = Sequential()

model.add(Conv2D(128, (3,3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(X, y, batch_size=48, epochs=10, validation_split=0.3, callbacks=[tensorboard])

3-conv-128-nodes-1-dense-1618846287
Epoch 1/10
50/50 [==============================] - 52s 1s/step - loss: 0.6341 - accuracy: 0.6942 - val_loss: 0.5642 - val_accuracy: 0.7463
Epoch 2/10
50/50 [==============================] - 52s 1s/step - loss: 0.5968 - accuracy: 0.7123 - val_loss: 0.5651 - val_accuracy: 0.7414
Epoch 3/10
50/50 [==============================] - 52s 1s/step - loss: 0.5996 - accuracy: 0.7085 - val_loss: 0.5647 - val_accuracy: 0.7444
Epoch 4/10
50/50 [==============================] - 53s 1s/step - loss: 0.5873 - accuracy: 0.7190 - val_loss: 0.5757 - val_accuracy: 0.7326
Epoch 5/10
50/50 [==============================] - 53s 1s/step - loss: 0.5806 - accuracy: 0.7144 - val_loss: 0.5465 - val_accuracy: 0.7453
Epoch 6/10
50/50 [==============================] - 53s 1s/step - loss: 0.5658 - accuracy: 0.7228 - val_loss: 0.5565 - val_accuracy: 0.7316
Epoch 7/10
50/50 [==============================] - 54s 1s/step - loss: 0.5647 - accuracy: 0.7144 - val_loss: 0.5486 - val_a

### Saving

In [11]:
model.save('bcd_model.h5')

### Testing


In [21]:
import cv2
import tensorflow as tf

CATEGORIES = ["Cancer", "Normal"]

def prepare(filepath):
    IMG_SIZE = 100
    img_arr = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
    return new_arr.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

model = tf.keras.models.load_model('bcd_model.h5')

In [22]:
prediction = model.predict([prepare('test_samples/Cancer/c_2.png')])
print(CATEGORIES[int(prediction[0][0])])
print(int(prediction[0][0]))

Cancer
0


In [23]:
prediction = model.predict([prepare('test_samples/Normal/n_1.png')])
print(CATEGORIES[int(prediction[0][0])])
print(int(prediction[0][0]))

Normal
1
